In [ ]:
import pandas as pd
from databaker.framework import *
from databakerUtils.writers import v4Writer
import glob, math
from databakerUtils.api import getAllCodes, getAllLabels

timeOfData = '2018'

locationTable9 = 'table_9/' + timeOfData + '/*'
locationTable10 = 'table_10/' + timeOfData + '/*'
output_file = 'v4_ashe_table_9and10_' + timeOfData + '.csv'

#all files in location
allFiles9 = glob.glob(locationTable9)
allFiles10 = glob.glob(locationTable10)
allFiles = allFiles9 + allFiles10

#ignoring any files that are not ashe files and ignoring gender pay gap file
allFiles = [file for file in allFiles if '.12' not in file]

#separate data and CV interval data
files = [file for file in allFiles if not file.endswith('CV.xls')]
filesCV = [file for file in allFiles if file.endswith('CV.xls')]

#making sure both lists are in the same order
files = sorted(files)
filesCV = sorted(filesCV)

#loading in all tabs for data
allTabs = []
for file in files:
    readFile = loadxlstabs(file)
    allTabs.append(readFile)

#loading in all tabs for CV interval data
allTabsCV = []
for file in filesCV:
    readFile = loadxlstabs(file)
    allTabsCV.append(readFile)
    
#above process creates a list of lists
#need to flatten the lists    
flatList = [item for subitem in allTabs for item in subitem]
flatListCV = [item for subitem in allTabsCV for item in subitem]

#removing the info tabs from each spreadsheet
tabs = [tab for tab in flatList if tab.name != 'Notes']
tabsCV = [tab for tab in flatListCV if tab.name != 'CV notes']

#quick check to make sure number of files or number of tabs hasn't changed
if len(tabs) == len(tabsCV) != len(files)*9:
    raise Exception('Number of files or number of tabs has changed')

'''will be iterating the databaking process'''
#max number of rows out of all the sheets
maxLength = []
for tab in tabs:
    tabMax = len(tab.excel_ref('A'))
    maxLength.append(tabMax)
maxLength = max(maxLength)
batchNumber = 20    #iterates over this many rows at a time
numberOfIterations = math.ceil(maxLength/batchNumber)   #databaking will iterate this many times

In [ ]:
'''Functions'''

def pconGeography(value):
    '''
    Changes some of the geography codes to match the admin codes
    (top levels weren't included in geography hierarchy provided)
    '''
    lookup = {
            'E12000001':'E15000001',
            'E12000002':'E15000002',
            'E12000003':'E15000003',
            'E12000004':'E15000004',
            'E12000005':'E15000005',
            'E12000006':'E15000006',
            'E12000007':'E15000007',
            'E12000008':'E15000008',
            'E12000009':'E15000009'
            }
    return lookup.get(value,value)

def renameGeog(value):
    '''
    geography label of "East" is used in dataset but "Eastern" used in codelist
    '''
    lookup = {
            'East':'Eastern'
            }
    return lookup.get(value,value)

#pull in codelist for sheetName (ashe-earnings)
sheetNameURL = 'https://api.beta.ons.gov.uk/v1/code-lists/ashe-earnings/editions/one-off/codes'
dataSheetNameCodes = getAllCodes(sheetNameURL)
dataSheetNameLabels = getAllLabels(sheetNameURL)
sheetNameDict = dict(zip(dataSheetNameLabels,dataSheetNameCodes))

def sheetNameLookup(value):
    '''returns ashe-earnings labels from sheetName'''
    value = '.'+value.split('.')[1]
    lookup = {
            '.1a':'Weekly pay - Gross',
            '.2a':'Weekly pay - Excluding overtime',
            '.3a':'Basic pay - Including other pay',
            '.4a':'Overtime pay',
            '.5a':'Hourly pay - Gross',
            '.6a':'Hourly pay - Excluding overtime',
            '.7a':'Annual pay - Gross',
            '.8a':'Annual pay - Incentive',
            '.9a':'Paid hours worked - Total',
            '.10a':'Paid hours worked - Basic',
            '.11a':'Paid hours worked - Overtime'
            }
    return lookup[value]

def sheetNameCodeLookup(value):
    '''returns ashe-earnings codes from labels'''
    return sheetNameDict.get(value,value.lower().replace(' - ','-').replace(' ','-'))

def tableNumberLookup(value):
    lookup = {
            '9':'Workplace',
            '10':'Residence'
            }
    return lookup[value]


#pull in codelist for 'variable' (ashe-statistics)
variableURL = 'https://api.beta.ons.gov.uk/v1/code-lists/ashe-statistics/editions/one-off/codes'
dataVariableCodes = getAllCodes(variableURL)
dataVariableLabels = getAllLabels(variableURL)
variableDict = dict(zip(dataVariableLabels,dataVariableCodes))

def variableTypeCodeLookup(value):
    '''returns ashe-statistics code from label'''
    return variableDict.get(value,value)

def variableType(value):
    #one of these lookups needs removing
    '''returns variable labels in a more useable format (string) also matches labels'''
    lookup = {
            '10.0':'10th percentile', 
            '20.0':'20th percentile', 
            '25.0':'25th percentile', 
            '30.0':'30th percentile',
            '40.0':'40th percentile', 
            '60.0':'60th percentile', 
            '70.0':'70th percentile', 
            '75.0':'75th percentile', 
            '80.0':'80th percentile', 
            '90.0':'90th percentile'
            }
    return lookup.get(value,value)

#splitting tabName into sex and working pattern

def sexLabels(value):
    '''returns ashe-sex labels from tabName'''
    lookup = {
            'Full-Time':'All', 
            'Part-Time':'All',
            'Male Full-Time':'Male', 
            'Male Part-Time':'Male', 
            'Female Full-Time':'Female',
            'Female Part-Time':'Female'
            }
    return lookup.get(value,value)

def sexCodes(value):
    '''returns ashe-sex codes from labels'''
    return value.lower()

def workingPatternLabels(value):
    '''returns working patterns labels from tabName'''
    lookup = {
            'Male':'All', 
            'Female':'All',
            'Male Full-Time':'Full-Time', 
            'Male Part-Time':'Part-Time', 
            'Female Full-Time':'Full-Time',
            'Female Part-Time':'Part-Time'
            }
    return lookup.get(value,value)

def workingPatternCodes(value):
    '''returns working pattern codes from labels'''
    return value.lower()

In [ ]:
'''DataBaking'''
print('DataBaking...')
conversionsegments = []

for i in range(0,numberOfIterations):

    Min = str(6 + batchNumber * i)  #data starts on row 6
    Max = str(int(Min) + batchNumber - 1)

    for tab in tabs:
        
        #columns are named badly
        #quick check to make sure they haven't changed
        if tab.excel_ref('C5').value != '(thousand)':
            raise Exception("Column names aren't right")
            
        if tab.excel_ref('S7').value != 'Key':
            raise Exception('Key has moved')
            
        key = tab.excel_ref('S7').expand(RIGHT).expand(DOWN)    #referenced but not used
        junk = tab.excel_ref('A').filter(contains_string('Northern Ireland')).shift(DOWN).expand(DOWN)
        
        geographyNames = tab.excel_ref('A'+Min+':A'+Max).is_not_blank().is_not_whitespace() - junk
        geographyCodes = tab.excel_ref('B'+Min+':B'+Max).is_not_blank().is_not_whitespace()
        
        #ignoring the annual percentage change and number of jobs
        columnsToIgnore = tab.excel_ref('E') | tab.excel_ref('G') | tab.excel_ref('C')
        variable = tab.excel_ref('C5').expand(RIGHT).is_not_blank().is_not_whitespace() - columnsToIgnore
        
        tabName = tab.name
        
        sheetName = tab.excel_ref('a1').value.split(' ')[1]
    
        tableNumber = sheetName.split('.')[0]
    
        obs = variable.waffle(geographyNames)
        
        dimensions = [
                HDimConst(TIME, timeOfData),
                HDim(geographyCodes, GEOG, DIRECTLY, LEFT),
                HDim(geographyNames, 'GeogNames', DIRECTLY, LEFT),
                HDim(variable, 'Variable', DIRECTLY, ABOVE),
                HDimConst('tabName', tabName),
                HDimConst('sheetName', sheetName),
                HDimConst('tableNumber', tableNumber)
                ]
        
        if len(obs) != 0:
            #only use ConversionSegment if there is data
            conversionsegment = ConversionSegment(tab, dimensions, obs).topandas()
            conversionsegments.append(conversionsegment)
        
    if (int(Max)-1)%10 == 0:
        #return a message after every round (iteration number)
        print('Round {} out of {} done.. rows between {} and {}'.format(i+1,numberOfIterations,Min,Max))
        
    
data = pd.concat(conversionsegments)
df = v4Writer(output_file,data,asFrame=True)

firstTime = str(datetime.datetime.now() - startTime)
print("Time taken: " + firstTime) 
 
'''DataBaking CV interval data'''
print('DataBaking the CV intervals...')

conversionsegments = []

for i in range(0,numberOfIterations):

    Min = str(6 + batchNumber * i)  #data starts on row 6
    Max = str(int(Min) + batchNumber - 1)
  
    for tab in tabsCV:
        
        #columns are named badly
        #quick check to make sure they haven't changed
        if tab.excel_ref('C5').value != '(thousand)':
            raise Exception("Column names aren't right")
            
        if tab.excel_ref('S7').value != 'Key':
            raise Exception('Key has moved')
            
        key = tab.excel_ref('S7').expand(RIGHT).expand(DOWN)    
        junk = tab.excel_ref('A').filter(contains_string('Northern Ireland')).shift(DOWN).expand(DOWN)
        
        geographyNames = tab.excel_ref('A'+Min+':A'+Max).is_not_blank().is_not_whitespace() - junk
        geographyCodes = tab.excel_ref('B'+Min+':B'+Max).is_not_blank().is_not_whitespace()
        
        #ignoring the annual percentage change and number of jobs
        columnsToIgnore = tab.excel_ref('E') | tab.excel_ref('G') | tab.excel_ref('C')
        variable = tab.excel_ref('C5').expand(RIGHT).is_not_blank().is_not_whitespace() - columnsToIgnore
        
        tabName = tab.name
        
        sheetName = tab.excel_ref('a1').value.split(' ')[1]
    
        tableNumber = sheetName.split('.')[0]
    
        obs = variable.waffle(geographyNames)
        
        dimensions = [
                HDimConst(TIME, timeOfData),
                HDim(geographyCodes, GEOG, DIRECTLY, LEFT),
                HDim(geographyNames, 'GeogNames', DIRECTLY, LEFT),
                HDim(variable, 'Variable', DIRECTLY, ABOVE),
                HDimConst('tabName', tabName),
                HDimConst('sheetName', sheetName),
                HDimConst('tableNumber', tableNumber)
                ]
        
        if len(obs) != 0:
            #only use ConversionSegment if there is data
            conversionsegment = ConversionSegment(tab, dimensions, obs).topandas()
            conversionsegments.append(conversionsegment)
        
    if (int(Max)-1)%10 == 0:
        #return a message after every round (iteration number)
        print('Round {} out of {} done.. rows between {} and {}'.format(i+1,numberOfIterations,Min,Max))
        
dataCV = pd.concat(conversionsegments)
dfCV = v4Writer(output_file,dataCV,asFrame=True) 

#quick check to make sure data and CV data is same length
if len(df.index) != len(dfCV.index):
    raise Exception('Data and CV interval data lengths don\'t match')

In [ ]:
'''Post processing'''

#V4 column for dfCV is the CV intervals for data
df['CV'] = dfCV['V4_1']

df['Time_codelist'] = df['Time']
df['Geography'] = df['GeogNames']
df = df.drop(['GeogNames', 'GeogNames_codelist'], axis = 1)

#renaming columns
colsRename = {
        'V4_1':'V4_2',
        'Time':'time',
        'Time_codelist':'calendar-years',
        'Geography':'geography',
        'Geography_codelist':'parliamentary-geography',
        'Variable':'statistics',
        'Variable_codelist':'ashe-statistics',
        'sheetName':'hoursandearnings',
        'sheetName_codelist':'ashe-hours-and-earnings',
        'tableNumber':'workplaceorresidence',
        'tableNumber_codelist':'ashe-workplace-or-residence'
        }

df['Geography'] = df['Geography'].apply(lambda x:x.strip())
df['Geography'] = df['Geography'].apply(renameGeog)
df['Geography_codelist'] = df['Geography_codelist'].apply(pconGeography)

df['sheetName'] = df['sheetName'].apply(sheetNameLookup)
df['sheetName_codelist'] = df['sheetName'].apply(sheetNameCodeLookup)
df['sheetName_codelist'] = df['sheetName_codelist'].apply(lambda x:x.replace(' ','-'))

df['tableNumber'] = df['tableNumber'].apply(tableNumberLookup)
df['tableNumber_codelist'] = df['tableNumber'].apply(lambda x:x.lower())

df['Variable'] = df['Variable'].apply(variableType)
df['Variable_codelist'] = df['Variable'].apply(variableTypeCodeLookup)

df['tabName_codelist'] = df['tabName'].apply(lambda x:x.lower().replace('-','_'))

df['sex'] = df['tabName'].apply(sexLabels)
df['ashe-sex'] = df['sex'].apply(sexCodes)

df['workingpattern'] = df['tabName'].apply(workingPatternLabels)
df['ashe-working-pattern'] =df['workingpattern'].apply(workingPatternCodes)

#reordering columns
df = df[['V4_1', 'Data_Marking', 'CV', 'Time_codelist', 'Time',
         'Geography_codelist', 'Geography', 'Variable_codelist', 'Variable',
         'ashe-sex', 'sex', 'ashe-working-pattern', 'workingpattern', 
         'sheetName_codelist', 'sheetName', 'tableNumber_codelist', 'tableNumber']]

df = df.rename(columns = colsRename)

#data markers for CV's need to be filled in
df.loc[df['CV'] == '','CV'] = 'x'

df.to_csv(output_file, index = False)